### Libraries

In [1]:
import pandas as pd
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, FactorRange, Legend
from bokeh.plotting import figure

import seaborn as sns

### Importing the Crime Data

In [2]:
data = pd.read_csv('Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')

### Filter the Data for the Period 2010-2018

In [3]:
data = data[data['Date'].str.strip().str[-4:].isin(['2010','2011','2012','2013', '2014', 
                                                    '2015','2016','2017','2018'])]

### Focus Crimes

In [4]:
focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])
data = data[data['Category'].isin(focuscrimes)]

### Creating the Hour Column

In [5]:
data['Hour'] = data['Time'].str.strip().str[0:2]

hours = ['Midnight - 1am', '1am - 2am', '2am - 3am', '3am - 4am', '4am - 5am', '5am - 6am', '6am - 7am', '7am - 8am', '8am - 9am', '9am - 10am', '10am - 11am', '11am - Noon', 'Noon - 1pm', '1pm - 2pm', '2pm - 3pm', '3pm - 4pm', '4pm - 5pm', '5pm - 6pm', '6pm - 7pm', '7pm - 8pm', '8pm - 9pm', '9pm - 10pm', '10pm - 11pm', '11pm - Midnight']
temp_hour = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

for i in range(9):
    data['Hour'] = data['Hour'].str.replace(temp_hour[i], hours[i])    
data['Hour'] = data['Hour'].str.replace('09', '%temp1%').replace('10', '%temp2%').replace('11', '11am - Noon').replace('%temp2%', '10am - 11am').replace('%temp1%', '9am - 10am')
for i in range(12):
    data['Hour'] = data['Hour'].str.replace(temp_hour[i+12], hours[i+12])

### Create the Hourly DataFrame

In [6]:
hourData = pd.pivot_table(data, index = "Hour", columns = "Category",values = 'IncidntNum' ,aggfunc = 'count')
hourData = hourData.reindex(hours)

### Normalized Data

In [7]:
normal = hourData.copy()
normal = normal.astype(float)

for i in range(hourData.shape[1]):
    for j in range(hourData.shape[0]):      
        normal.iloc[:,i][j] = hourData.iloc[:,i][j] / sum(hourData.iloc[:,i])

### Convert Pandas DataFrame to Bokeh ColumnDataSource

In [8]:
source = ColumnDataSource(normal)

### Output Notebook

In [9]:
output_notebook()

Loading BokehJS ...

### Create an Empty Figure

In [10]:
p = figure(x_range=FactorRange(factors=normal.index),
           plot_width=1200,
           title='Crimes per Hour', 
           x_axis_label='Hour of the Day', 
           y_axis_label='Relative Frequency')

### Different Colors for Different Crime

In [11]:
cmap = sns.color_palette('icefire', len(normal.columns)).as_hex()

### Add the Bars

In [12]:
bar = {}
items = []

for i, crime in enumerate(focuscrimes):
    bar[crime] = p.vbar(x='Hour', 
                    top=crime, 
                    source=source, 
                    width=0.7,
                    color=cmap[i],
                    fill_alpha=1.5,
                    muted=True, 
                    muted_alpha=0.005) 
    items.append((crime, [bar[crime]]))

### Figure Settings

In [13]:
p.xaxis.major_label_orientation = 1
p.y_range.start = 0

### Create an Interactive Legend

In [14]:
legend = Legend(items=items)
p.add_layout(legend, 'left')    
p.legend.click_policy = 'mute'

show(p)